In [139]:
import csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import re
import matplotlib.gridspec as gridspec

#matplotlib.style.use('ggplot')

# Event files pre-processing

In [34]:
def modifierfun (row):
    if (pd.notnull(row['Modifier_1'])):
        return row['Modifier_1'];
    if (pd.notnull(row['Modifier_2'])):
        return row['Modifier_2'];
    if (pd.notnull(row['Modifier_3'])):
        return row['Modifier_3'];

def get_sec(row):
    s=row['Time_Absolute_hms']
    l = s.split(':')
    return int(l[0]) * 3600 + int(l[1]) * 60 + int(l[2])

## Generate event related dataframe from ObserverXT data

In [35]:
dfAll=pd.DataFrame()
for fn in os.listdir('./../observerXT/'):
    l=fn.split(' - ')
    t=l[1].split('_')
    sujet=int(t[0][1:])
    matnum=t[1][-1]
    matname=t[2]    
    niveau={
        'debutant':1,
        'amateur':2,
        'semipro':3,
        'pro':4,
        'champion':5,
        'legend':6
    }[matname]
    eventfile='./../observerXT/'+fn
    #print eventfile
    df=pd.read_csv(eventfile,sep=';', quoting=csv.QUOTE_ALL,encoding='utf-16')
    df = df[df.Event_Type == "State start"]
    df['Absolute_sec']=df.apply (lambda row: get_sec(row),axis=1)
    df['annovalue']=df.apply (lambda row: modifierfun(row),axis=1)
    df=df[['Time_Absolute_hms','Time_Relative_hmsf','Absolute_sec','Behavior','annovalue']]
    
    ndf=df.pivot(index='Absolute_sec', columns='Behavior', values='annovalue')
    ndf=ndf[['event','emotions','arousal','valence']]
    ndf=ndf.reset_index()
    ndf.loc[:,'sujet']=sujet
    ndf.loc[:,'match']=matnum
    ndf.loc[:,'niveau']=niveau
    dfAll=dfAll.append(ndf,ignore_index=True)
dfAll=dfAll.sort(['sujet', 'match','Absolute_sec'], ascending=[1,1,1])
dfAll['sujet']=dfAll['sujet'].astype(int)
#dfAll.info() #datatype of each column
dfAll=dfAll.reset_index(drop=True)
dfAll.to_csv('./../Allevent.txt',encoding='utf-16')

## Analyse event emotion dataframe

#### Number of event ####

In [ ]:
# dfAll.pivot_table(values='event',index='sujet',columns='match',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of events')
plt.title('Number of event annoted by each subject')
plt.show()

#### Emotion / participant

In [123]:
dfAll.pivot_table(values='event',index='sujet',columns='emotions',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of emotions')
plt.title('Number of emotions annoted by each subject')
plt.show()

In [129]:
dfAll.pivot_table(values='event',index='sujet',columns='arousal',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of emotion - arousal')
plt.title('Number of emotion - arousal annoted by each subject')
plt.show()

In [10]:
dfAll.pivot_table(values='event',index='sujet',columns='valence',aggfunc=len).plot(stacked=True,kind='barh',fontsize=8)
plt.ylabel('subject')
plt.xlabel('Number of emotion -valence')
plt.title('Number of emotion - valence annoted by each subject')
plt.show()

#### Emotion / all

In [133]:
dfAll[['emotions','event']].groupby(['emotions']).count().plot(kind='bar',rot=30,legend=False)
plt.ylabel('Number')
plt.xlabel('emotion')
plt.title('Number of emotion annoted by each subject')
plt.show()

In [134]:
dfAll[['arousal','event']].groupby(['arousal']).count().plot(kind='bar',legend=False)
plt.ylabel('Number')
plt.xlabel('Arousal')
plt.title('Number of emotion (arousal) annoted by each subject')
plt.show()

In [135]:
dfAll[['valence','event']].groupby(['valence']).count().plot(kind='bar',legend=False)
plt.ylabel('Number')
plt.xlabel('emotion')
plt.title('Number of emotion (valence) annoted by each subject')
plt.show()

In [36]:
dfAll

Behavior,Absolute_sec,event,emotions,arousal,valence,sujet,match,niveau
0,47637,rate,frustration,2,-1,1,1,2
1,47813,arret du gardien,colere,2,-1,1,1,2
2,47852,but,colere,2,-3,1,1,2
3,48157,tir,peur,1,-1,1,1,2
4,49632,arret du gardien,frustration,2,-2,1,2,1
5,49661,tir,peur,1,-1,1,2,1
6,49706,rate,frustration,1,-1,1,2,1
7,50038,faute,neutre,0,0,1,2,1
8,50049,but,colere,2,-2,1,2,1
9,50134,rate,frustration,2,-1,1,2,1


#### Distribution & scatter plot (Arousal - Valence)

In [37]:
tmp=dfAll.as_matrix(columns=['arousal','valence'])
dfAll[dfAll.isnull().any(axis=1)]

Behavior,Absolute_sec,event,emotions,arousal,valence,sujet,match,niveau


In [75]:
import numpy as np
import numpy.random
import matplotlib.pyplot as plt
tmp=dfAll.as_matrix(columns=['arousal','valence'])
x = np.int_(tmp[:,0])
y = np.int_(tmp[:,1])
heatmap, xedges, yedges = np.histogram2d(x, y,bins=[10,10])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap, extent=extent, clim=(0.0, 170))
plt.show()

In [88]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

# the random data
tmp=dfAll.as_matrix(columns=['arousal','valence'])
x = np.int_(tmp[:,0])
y = np.int_(tmp[:,1])
heatmap, xedges, yedges = np.histogram2d(x, y,bins=[7,7])
extent = [-3.5, 3.5, -3.5, 3.5]

nullfmt = NullFormatter()         # no labels

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]

rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
plt.figure(1, figsize=(8, 8))


axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

# no labels
axHistx.xaxis.set_major_formatter(nullfmt)
axHisty.yaxis.set_major_formatter(nullfmt)

# the scatter plot:
axScatter.imshow(heatmap, extent=extent, cmap="hot")


# now determine nice limits by hand:
binwidth = 1
xymax = 3.5
lim = 3.5

axScatter.set_xlim((-lim, lim))
axScatter.set_ylim((-lim, lim))

bins = np.arange(-lim, lim + binwidth, binwidth)
axHistx.hist(x, bins=bins)
axHisty.hist(y, bins=bins, orientation='horizontal')

axHistx.set_xlim(axScatter.get_xlim())
axHisty.set_ylim(axScatter.get_ylim())

plt.show()

## Select event files for analysis

In [ ]:
selectdf=dfAll[dfAll.valence>0]
for i in range(len(selectdf)):
    chunk=selectdf.iloc[i,5]+'_'+str(selectdf.index[i])+'.txt'
    print chunk


In [14]:

#chunk='Data/match1.txt'
chunk='S1_40.txt'
chunkdf=pd.read_csv(chunk,sep='\t',nrows=30000,header=None,usecols=range(1,12),names=['rsp', 'ppg', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
#chunkdf=pd.read_csv(chunk,sep='\t',header=None,nrows=60000*13.8,usecols=range(1,14),names=['rsp', 'ppg', 'HR', 'x', 'emg_s','skt','RR','y','eda','ecg','z','emg_f','temp'])



## Generate event related signal sequence from Biopac data

In [ ]:
biopacfilels=[]
for fn in os.listdir('./../biopac/'):
    biopacfilels.append(fn)
#tobiifilels=[]
#for fn in os.listdir('./../tobii/files'):
    #tobiifilels.append(fn)
for i in range(len(dfAll)):
    #print i
    eventpoint=dfAll.iloc[i,0]
    ebegin=eventpoint-15
    eend=eventpoint+15
    sujet=dfAll.iloc[i,5]
    eidx = dfAll.index[i]
    #Biopacfile
    patternbiopacfile="^S"+str(sujet)+"_.*"
    regexb=re.compile(patternbiopacfile)
    bfilel=[m.group(0) for l in biopacfilels for m in [regexb.search(l)] if m]
    if len(bfilel)==1:
        bfile = './../biopac/'+bfilel[0]
        efile = './../biopacEvent/S_'+str(sujet)+'_'+str(eidx)+"_.txt"
        cmd_biopac = "sed -n '/^" + str(ebegin) + "/,/^" + str(eend) + "/p' "+bfile+">"+efile
        #print cmd_biopac
        os.system(cmd_biopac)
    #Tobiifile
    
    #Facereaderfile
    
       

# Match Journal processing

In [120]:
def get_sec(row,col):
    s=row[col]
    l = s.split(':')
    return int(l[0]) * 3600 + int(l[1]) * 60 + int(l[2])

Mdf={}
Datapath='./../Matchjrn/'
Matchfilepath='./../biopacMatch/'
Biofilepath="./../biopac/"
biofilelist=[]
for fn in os.listdir(Biofilepath):
    biofilelist.append(fn)

num=0
for fn in os.listdir(Datapath):
    num=num+1
    sujet = re.split(r'\s*[.\-\s]\s*', fn)[1]  # M - S2.csv
    print num, sujet
    matchfile = Datapath + fn
    #print matchfile
    Mdfname=sujet
    df=pd.read_csv(matchfile,sep=',',header=None,index_col=0)
    df[3]=df.apply (lambda row: get_sec(row,1),axis=1)
    df[4]=df.apply (lambda row: get_sec(row,2),axis=1)
    df[5]=df[4]-df[3]
    df=df.loc[['base0','en1','en2','base1','m11','m12','base2','m21','m22','base3','m31','m32'],:]
    df.loc[['base0','base1','base2','base3'],6]=df.loc[:,4]-180
    df.loc[['en1','en2','m11','m12','m21','m22','m31','m32'],6]=df.loc[:,4]-240
    df[6]=df[6].astype(int)
    #df_o=df[[6,4]]
    Mdf[Mdfname]=df
    #df=pd.read_csv(matchfile,sep=',',header=None,index_col=0)
    for i in range(len(df)):
        mfile=Matchfilepath+sujet+'_'+str(df.index[i])+'.txt'
        #print mfile
        mbegin=df.ix[i,6]
        mend=df.ix[i,4]
        for j in biofilelist:
            if j.split('_')[0]==sujet:
                biopacfile=j
        #cmd = "sed -n '/^" + str(mbegin) + "/,/^" + str(mend) + "/p' "+"./../biopac/"+ biopacfile + ">"+mfile
        #print cmd
        #os.system(cmd)

In [214]:
Mdf['S4'] #Mdf['S11'].ix['base0',6]

,1,2,3,4,5,6
base0,16:30:30,16:34:10,59430,59650,220,59470
en1,16:34:11,16:40:25,59651,60025,374,59785
en2,16:43:40,16:48:34,60220,60514,294,60274
base1,16:53:07,16:56:17,60787,60977,190,60797
m11,16:56:27,17:01:40,60987,61300,313,61060
m12,17:03:52,17:09:03,61432,61743,311,61503
base2,17:27:00,17:30:24,62820,63024,204,62844
m21,17:30:34,17:35:43,63034,63343,309,63103
m22,17:38:09,17:43:54,63489,63834,345,63594
base3,17:58:00,18:01:17,64680,64877,197,64697


# Participant informations

In [124]:
participant=pd.read_csv('./../MatchStatistiques.csv',sep=',')


# Plot the whole sequence

In [243]:
def ploteventtrait(plt, eventtmp):
    m= (ymin+ymax)/2
    span= ymax-ymin
    for index, row in eventtmp.iterrows():
        mywidth=np.fabs(row['arousal'])+1
        if row['arousal']>=0:
            pos_down = 0.6
            pos_up= 1
        else:
            pos_down = 0
            pos_up= 0.4
        plt.axvline(row['rela_point']-700,color='r',ymin=pos_down, ymax=pos_up,linewidth=mywidth)
        
    for index, row in eventtmp.iterrows():
        mywidth=np.fabs(row['valence'])+1
        if row['valence']>=0:
            pos_down = 0.6
            pos_up= 1
        else:
            pos_down = 0
            pos_up= 0.4
        plt.axvline(row['rela_point']+700,color='g',ymin=pos_down, ymax=pos_up,linewidth=mywidth)   
    

In [271]:
def plotseq(chunkdf,sujet, sequence,eventtmp):
    fig = plt.figure(figsize=(30,20))
    figurename='S'+str(sujet)+'_'+sequence
    fig.suptitle(figurename)
    gs = gridspec.GridSpec(7,1)
     # raw signal

    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(range(l), chunkdf['ecg'], linewidth=1, label='ecg')
    #ax1.legend()
    ax1.set_ylabel('ECG')
    ploteventtrait(plt,eventtmp)

    ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
    ax2.plot(range(l), chunkdf['rsp'], linewidth=1, label='respiration')
    #ax2.legend()
    ax2.set_ylabel('Resp')
    ploteventtrait(plt,eventtmp)

    ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
    ax3.plot(range(l), chunkdf['eda'], linewidth=1, label='eda')
    #ax3.legend()
    ax3.set_ylabel('EDA')
    ploteventtrait(plt,eventtmp)

    ax4 = fig.add_subplot(gs[3, 0], sharex=ax1)
    ax4.plot(range(l), chunkdf['skt'], linewidth=1, label='skinTemp')
    #ax4.legend()
    ax4.set_ylabel('SkinTemp')
    ploteventtrait(plt,eventtmp)

    ax5 = fig.add_subplot(gs[4, 0], sharex=ax1)
    ax5.plot(range(l), chunkdf['emg_s'], linewidth=1, label='emg_s')
    #ax5.legend()
    ax5.set_ylabel('EMG_m')
    ploteventtrait(plt,eventtmp)

    ax6 = fig.add_subplot(gs[5, 0], sharex=ax1)
    ax6.plot(range(l), chunkdf['emg_f'], linewidth=1, label='emg_f')
    #ax6.legend()
    ax6.set_ylabel('EMG_f')
    ploteventtrait(plt,eventtmp)

    ax7 = fig.add_subplot(gs[6, 0], sharex=ax1)
    ax7.plot(range(l), chunkdf[['x','y','z']], linewidth=1)
    #ax7.legend(['x','y','z'])
    ax7.set_ylabel('Acce')
    ploteventtrait(plt,eventtmp)

    savename='./../plot/'+figurename+'.pdf'
    fig.savefig(savename)
    #plt.show()
    plt.close()

In [268]:
sujet = 33
sequence = 'm12'
bioMatchPath="./../biopacMatch/"
#Read Biopac files of selected sequence : exp: ./../biopacMatch/S4_m12.txt
matchFilename=bioMatchPath+"S"+str(sujet)+"_"+sequence+".txt"
print matchFilename
chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=240000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
l=len(chunkdf)

#Get beginning timestamp (Absolute_sec) of selected sequence
strsujet = 'S'+str(sujet)
begin = Mdf[strsujet].ix[sequence,6]
end = Mdf[strsujet].ix[sequence,4]

#Get event list of selected sequence
eventtmp=dfAll.ix[(dfAll['sujet']==sujet) ,:]
eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]
eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp
#eventtmp
plotseq(chunkdf,33,'m12',eventtmp)

./../biopacMatch/S33_m12.txt


#### Draw all

In [270]:
bioMatchPath="./../biopacMatch/"
for fn in os.listdir(bioMatchPath):
    sujet=re.split('_|\.|S', fn)[1]
    sequence=re.split('_|\.|S', fn)[2]
    matchFilename=bioMatchPath+fn
    #print matchFilename
    chunkdf=pd.read_csv(matchFilename,sep='\t',nrows=240000,header=None,usecols=range(0,11),names=['ts','rsp', 'x', 'emg_s','skt','y','eda','ecg','z','emg_f','temp'])
    l=len(chunkdf)
    #Get beginning timestamp (Absolute_sec) of selected sequence
    strsujet = 'S'+str(sujet)
    begin = Mdf[strsujet].ix[sequence,6]
    end = Mdf[strsujet].ix[sequence,4]

    #Get event list of selected sequence
    eventtmp=dfAll.ix[(dfAll['sujet']==int(sujet)) ,:]
    eventtmp=eventtmp[(eventtmp.Absolute_sec>begin)&(eventtmp.Absolute_sec<end)]
    eventtmp['rela_point']=pd.Series((eventtmp.Absolute_sec-begin)*1000) #event point respect to the beginning timestamp
    #eventtmp
    #print matchFilename, len(eventtmp)
    plotseq(chunkdf,sujet,sequence,eventtmp)

/usr/lib/python2.7/dist-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


# Calculate features for each event file

## ECG

In [237]:
import numpy as np
from biosppy.signals import ecg
from biosppy.signals import tools as st

# load raw ECG signal
sig_ecg = chunkdf['ecg']

#heart rate
ts, filtered, rpeaks, templates_ts, templates, hr_ts,hr = ecg.ecg(signal=sig_ecg, sampling_rate=1000,show=True)

In [139]:
from scipy.interpolate import interp1d
def uni_hr(hr,hr_ts,ts):
    """resample and iterpolate to generate equal interval heart rate
    Require
    --------
    from scipy.interpolate import interp1d
    
    Parameter
    ---------
    hr : heart rate value
    hr_ts : heart rate time stamp
    ts : desired equal interval time stamp
    
    Returns
    -------
    hrn: equal interval heart rate
    """
    hrn=[None]*len(ts)
    hrn=np.array(hrn)
    hrn[ts >= hr_ts[-1]] = hr[-1]
    hrn[ts <= hr_ts[0]] = hr[0]
    last=np.nonzero(ts>=hr_ts[-1])
    le=list(last[0])[0]
    first=np.nonzero(ts<=hr_ts[0])
    fe=list(first[0])[-1]
    f = interp1d(hr_ts, hr)
    hrn[fe:le+1]=f(ts[range(fe,le+1)])
    return hrn

In [20]:
st.signal_stats(hr)

ReturnTuple(mean=75.053903895212898, median=71.186611582989059, max=92.284711432744814, var=210.41592517847701, std_dev=14.505720429488395, abs_dev=8760.3856375308897, kurtosis=9.88154468904554, skewness=2.657043597595585)

In [141]:
x = hr_ts
y = hr
f = interp1d(hr_ts, hr)
plt.plot(x, y, 'o', ts, uni_hr(hr,hr_ts,ts))
plt.legend(['data', 'linear'], loc='best')
plt.show()

In [39]:
#heart beat
diffbeat=np.diff(hr_ts)
st.signal_stats(diffbeat)

ReturnTuple(mean=0.99714533214285717, median=0.9769674333333338, max=2.0047546011904771, var=0.18819019806846182, std_dev=0.43380894189546371, abs_dev=5.3388220333333374, kurtosis=17.998041105752424, skewness=3.734098115145015)

In [7]:
#down sampling
def downsampling(signal,sampling_rate,new_rate):
    r=sampling_rate/new_rate
    n_signal=signal[range(0,len(signal),r)]
    rate=sampling_rate/(r*1.0)
    return {'sig':n_signal,'rate':rate}
    

In [180]:
ecgd=downsampling(sig_ecg,1000,40)
hr_s=ecgd['sig']
sampling_rate=ecgd['rate']

In [214]:
#frequence hr 
freqs,power=st.power_spectrum(hr_s, sampling_rate,decibel=False)
band1=st.band_power(freqs,power,(8,9),decibel=False)
band1
plt.plot(freqs,power)
plt.show()

ReturnTuple(avg_power=3.7588152673903034e-05)

In [221]:
#frequence diff(beat) sampling_rate=1
freqs,power=st.power_spectrum(np.diff(rpeaks), sampling_rate=1,decibel=False)
band1=st.band_power(freqs,power,(8,9),decibel=False)
plt.plot(freqs,power)
plt.show()

## EDA

In [36]:
import numpy as np
from biosppy.signals import eda
from biosppy.signals import ecg
from biosppy.signals import tools as st

# load raw EDA signal

sig_eda = chunkdf['eda']
edad=downsampling(sig_eda,1000,40)
signal=edad['sig']
sampling_rate=edad['rate']

#eda.eda(signal=signal, sampling_rate=sampling_rate,show=False)

In [40]:
aux, _, _ = st.filter_signal(signal=signal,
                                 ftype='butter',
                                 band='lowpass',
                                 order=4,
                                 frequency=5,
                                 sampling_rate=sampling_rate)

    # smooth
sm_size = int(0.75 * sampling_rate)
filtered, _ = st.smoother(signal=aux,
                              kernel='boxzen',
                              size=sm_size,
                              mirror=True)
onsets, peaks, amps = eda.kbk_scr(signal=filtered, sampling_rate=sampling_rate)

In [42]:
df = np.diff(signal)

# smooth
size = int(1. * sampling_rate)
df, _ = st.smoother(signal=df, kernel='bartlett', size=size, mirror=True)

# zero crosses
zeros, = st.zero_cross(signal=df, detrend=True)
if np.all(df[:zeros[0]] > 0):
    zeros = zeros[1:]
if np.all(df[zeros[-1]:] > 0):
    zeros = zeros[:-1]

# exclude SCRs with small amplitude
thr = 0.1 * np.max(df)

scrs, amps, ZC, pks = [], [], [], []
for i in xrange(0, len(zeros) - 1, 2):
    scrs += [df[zeros[i]:zeros[i + 1]]]
    aux = scrs[-1].max()
    if aux > thr:
        amps += [aux]
        ZC += [zeros[i]]
        ZC += [zeros[i + 1]]
        pks += [zeros[i] + np.argmax(df[zeros[i]:zeros[i + 1]])]

scrs = np.array(scrs)
amps = np.array(amps)
ZC = np.array(ZC)
pks = np.array(pks)
onsets = ZC[::2]

In [59]:
onsets

array([213])

In [60]:
plt.plot(range(len(signal)),np.array(signal))
plt.show()

In [54]:
len(signal)

1200